In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import load_from_disk
import re
import random
from datasets import Dataset, DatasetDict
import json

In [3]:
# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'title' : data["title"],
        "context" : data["context"],
        "question" : data["question"],
        "id" : data["id"],
        "answer_start" : data["answers"]["answer_start"][0],
        "answer_text" : data["answers"]["text"][0],
        "document_id" : data["document_id"]
    })
  df = pd.DataFrame(data_list)
  return df

In [4]:
# 기본 wikipedia 데이터셋 불러오기
def get_wiki(path):
  with open(path, "r") as f:
    wiki = json.load(f)

  df = pd.DataFrame(wiki)
  df = df.transpose()
  return df

In [5]:
train_path = "original_train_dataset_path"
wiki_path = "original_test_dataset_path"

In [6]:
train_df = dataset_to_df(load_from_disk(train_path)["train"])
wiki_df = get_wiki(wiki_path)

In [ ]:
valid_df = dataset_to_df(load_from_disk(train_path)["validation"])
len(valid_df)

240

In [ ]:
len(train_df)

3952

In [ ]:
len(wiki_df)

60613

In [ ]:
train_df.head()

,title,context,question,id,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,30,4개,47334


In [ ]:
wiki_df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [ ]:
len(wiki_df["text"].unique())

56737

In [ ]:
len(train_df["context"].unique())

3340

In [ ]:
len(valid_df["context"].unique())

235

# gpt

In [8]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
chat_api = "your_api_key"

# gpt를 이용한 증강
- 3200개의 데이터를 만드는데 cpu 기준 약 45분
- gpu를 쓸 수 없다고 함.

In [ ]:
train_df.head()

,title,context,question,id,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,30,4개,47334


In [ ]:
print(f"id 개수 : {len(train_df['id'])}")
print(f"unique한 id 개수 : {len(train_df['id'].unique())}")

id 개수 : 3952
unique한 id 개수 : 3952


In [10]:
# answer_text와 같은 단어들의 시작 인덱스를 구하는 코드
def get_answer_word_idx(context, answer):
  indices = []
  index = context.find(answer)

  while index != -1:
    indices.append(index)
    index = context.find(answer, index + 1)

  return indices

In [11]:
# gpt에서 반환해준 값을 리스트 형식으로 담을 수 있도록 해주는 함수
def extract_qa_data(blocks):
    questions = []
    answers = []
    q_as = []
    idxes = []

    for idx, block in enumerate(blocks):
        try:
            # 파이썬 코드 블록 내에 있는 JSON 형식 부분만 추출
            json_str = re.search(r'\{.*\}', block, re.DOTALL).group(0)

            # JSON 형식 문자열에서 작은따옴표를 큰따옴표로 변환
            json_str = json_str.replace("'", "\"")

            # JSON 형식 문자열을 딕셔너리로 변환
            data = json.loads(json_str)

            # 각 필드 추출
            questions.append(data.get("question", ""))
            answers.append(data.get("answer", ""))
            q_as.append(data.get("q_a", ""))

        except json.JSONDecodeError as e:
            idxes.append(idx)
            #print(f"JSONDecodeError at index {idx}: {e}")
            #print(f"Problematic block: {block}")

    return questions, answers, q_as, idxes

In [12]:
def find_sentence_indices(context, sentence):
    # 불필요한 공백과 끝에 붙은 구두점 또는 생략 기호(...) 제거
    clean_sentence = sentence.strip().rstrip('.,!?…')

    # sentence에 해당하는 패턴을 찾기 위한 정규 표현식 생성
    # 문장 끝에는 구두점이나 생략 기호(...)가 있을 수 있으므로 이를 반영
    pattern = re.escape(clean_sentence) + r'[.,!?…]*\s*'

    # 정규 표현식으로 매칭
    match = re.search(pattern, context)

    if match:
        start_index = match.start()  # 시작 인덱스
        end_index = match.end()      # 끝 인덱스
        return start_index, end_index
    else:
        return -1, -1  # 문장을 찾지 못한 경우

In [13]:
from tqdm import tqdm
from openai import OpenAI

In [ ]:
contexts = []
titles = []
questions = []
answer_texts = []
document_ids = []

q_a = []
few_sample_train_df = train_df.sample(n=2).reset_index(drop=True)

client = OpenAI(api_key = chat_api)

for i in tqdm(range(len(train_df)), desc="Processing rows"):
  context = train_df.loc[i]["context"]

  contexts.append(context)
  titles.append(train_df.loc[i]["title"])
  document_ids.append(train_df.loc[i]["document_id"])
  question = train_df.loc[i]['question']
  answer_text = train_df.loc[i]['answer_text']
  answer_start = train_df.loc[i]['answer_start']

  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},

          # few-shot 1
          {"role": "user", "content": f"{few_sample_train_df.loc[0]['context']}이라는 context가 있어. 이 context에 대한 질문은 {few_sample_train_df.loc[0]['question']}야. 정답은?"},
          {"role": "assistant", "content": f"{few_sample_train_df.loc[0]['answer_text']}"},

          # few-shot 2
          {"role": "user", "content": f"{few_sample_train_df.loc[1]['context']}이라는 context가 있어. 이 context에 대한 질문은 {few_sample_train_df.loc[1]['question']}야. 정답은?"},
          {"role": "assistant", "content": f"{few_sample_train_df.loc[1]['answer_text']}"},

          # real content
          {
              "role": "user",
              "content" : f"{context} 라는 문맥에서 질문을 하나 만들고, 그에 대한 정답을 알려줘\\\
              {question}이랑은 다른 질문이여야 해. 질문이랑 정답은 context에 그대로 나와있어야 해.\\\
              정답은 문맥에서 그대로 있어야해.\\\
              정답에 마침표는 빼줘. 정답은 문장이 아닌 명사 형태이고, 15자 이내가 되도록 해줘\\\
              그리고 질문과 정답이 있는 문맥의 문장도 알려줘. 이 문장에 끝은 문맥에 있는 것 그대로 들고와줘. \\\
              마침표면 마침표, 쉽표면 쉼표, 없으면 없이. 개행, 공백, 큰 따옴표, 작은 따옴표도 그대로.\\\
              question, answer, q_a를 키 값으로 하는 딕셔너리 형태로 반환해줘\\\
              파이썬 딕셔너리 형태로"
          }
      ]
  )
  q_a.append(completion.choices[0].message.content)
print(q_a)

In [ ]:
for q in q_a[:30]:
  print(q)
  print("=" * 50)

```python
{
    "question": "도스토옙스키는 어떤 사람의 도움으로 작품을 완성했나?",
    "answer": "아내 안나",
    "q_a": "그 즈음에는 이미 사물을 분간하지 못할 만큼 눈이 어두워져 있었고 도스토옙스키가 침대 누워 구술한 것을 아내 안나가 속기 하여 작품을 완성했다."
}
```
```python
{
    "question": "가이아가 우라노스에게 거세를 당하며 복수를 다짐하였을 때 만든 것은?",
    "answer": "낫",
    "q_a": "낫을 만든 가이아는 자식들을 불러 자신을 도와 아버지 우라노스에게 반란을 일으킨다면 1인자의 자리를 약속하였지만, 가이아의 말을 들은 자식들은 우라노스에 대한 두려움으로 선뜻 나서지 못하였다."
}
```
```python
{
    "question": "장면의 직업은 무엇이었나요",
    "answer": "동성상고 서무주임",
    "q_a": "1936년 11월 동성상업학교 박준호(朴準鎬) 교장이 별세하자 동성상고 서무주임으로 있던 장면은 11월 19일 천주교재단의 추천으로 동성상업학교 교장에 천거되어 취임하였다"
}
```
```python
{
    "question": "플레이어는 누구에게 용언을 사용할 수 있도록 훈련 받았는가?",
    "answer": "그레이비어드",
    "q_a": "플레이어는 스카이림의 가장 높은 산인 세계의 목구멍에서 은둔하는 수도자 그레이비어드들의 호출로 그곳에 가게 된다. 그레이비어드들은 플레이어가 용언을 사용할 수 있도록 훈련시키고 알두인을 저지하는 그들의 역할을 소개한다"
}
```
```python
{
    "question": "르베르디는 어린 시절 아버지 밑에서 무엇을 배우며 보냈나?",
    "answer": "글",
    "q_a": "어린 르베르디는 포도 농장을 운영하던 아버지 밑에서 글을 익히며 어린 시절을 보냈다."
}
```
```python
{
    "question": "윤치호

In [ ]:
# 각각의 리스트에 값을 넣어줌
questions, answers, q_as, idxes = extract_qa_data(q_a)
print(questions)
print(answers)
print(q_as)

ids = [f"mrc-2-{i:06}" for i in range(len(q_as))]

# idxes에 따른 행 제거
titles = [titles[i] for i in range(len(titles)) if i not in idxes]
contexts = [contexts[i] for i in range(len(contexts)) if i not in idxes]
document_ids = [document_ids[i] for i in range(len(document_ids)) if i not in idxes]

In [ ]:
# gpt로 증강된 데이터들을 데이터 프레임으로 생성
gpt_train_df = pd.DataFrame(
    {"title" : titles,
     "context" : contexts,
     "question" : questions,
     "answer_text" : answers,
     "document_id" : document_ids,
     "q_a_sentence" : q_as,
     "id" : ids
     }
)

In [18]:
# answer_start 조정
answer_starts = []
for i in range(len(gpt_train_df)):
  indexes = get_answer_word_idx(gpt_train_df.loc[i]['context'], gpt_train_df.loc[i]['answer_text'])
  if len(indexes) > 1:
    sen_indexes = find_sentence_indices(gpt_train_df.loc[i]['context'], gpt_train_df.loc[i]['q_a_sentence'])
    if sen_indexes[0] != -1:
      for idx in indexes:
        if idx in range(sen_indexes[0], sen_indexes[1]):
          indexes = int(idx)
          break
    else:
      indexes = int(indexes[0])
  elif len(indexes) != 0:
    indexes = indexes[0]

  if isinstance(indexes, list) and (len(indexes) != 1 and len(indexes) != 0):
    indexes = indexes[0]

  answer_starts.append(indexes)

gpt_train_df['answer_start'] = answer_starts

In [19]:
gpt_train_df.head()

,title,context,question,answer_text,document_id,q_a_sentence,id,answer_start
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,미국의 주를 대표하는 기관은?,상원,18293,미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 ...,mrc-2-000000,176
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 가장 큰 차이는 무엇인가?,통합,51638,현대적 인사조직관리와 근대 인사조직관리의 가장 큰 차이는 통합이다.,mrc-2-000001,397
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가,백성,5028,강희제는 1717년(강희 56년) 강희제는 “한 가지 일에 부지런하지 않으면 온 천...,mrc-2-000002,366
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,중국 북방 계통,34146,"이 작품은 높이 18cm의 작은 불감으로, 청동으로 불감과 불상을 만들고 그 위에 ...",mrc-2-000003,625
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,외함 안쪽의 금속제 그릇 개수는?,3점의 그릇,47334,그 안쪽에 납작한 금속제 원형 합 2점과 금속제 원통형 합 등 3점의 그릇이 봉안되...,mrc-2-000004,142


In [20]:
# answer_text의 값이 null인 것은 거르기
# 인덱스 재설정
gpt_train_df = gpt_train_df[gpt_train_df['answer_start'].apply(lambda x: x != [])]
gpt_train_df = gpt_train_df[gpt_train_df['answer_text'].apply(lambda x: x != [])]

gpt_train_df = gpt_train_df.reset_index(drop=True)

In [21]:
# anwer_start와 answer_text 합친 후 drop 해주는 코드
gpt_train_df['answers'] = gpt_train_df[['answer_start', 'answer_text']].apply(lambda x: {'answer_start': [(x['answer_start'])], 'text': [x['answer_text']]}, axis=1)
gpt_train_df = gpt_train_df.drop(columns=['answer_start', 'answer_text', 'q_a_sentence'])

In [22]:
gpt_train_df.head()

,title,context,question,document_id,id,answers
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,미국의 주를 대표하는 기관은?,18293,mrc-2-000000,"{'answer_start': [176], 'text': ['상원']}"
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 가장 큰 차이는 무엇인가?,51638,mrc-2-000001,"{'answer_start': [397], 'text': ['통합']}"
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가,5028,mrc-2-000002,"{'answer_start': [366], 'text': ['백성']}"
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,34146,mrc-2-000003,"{'answer_start': [625], 'text': ['중국 북방 계통']}"
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,외함 안쪽의 금속제 그릇 개수는?,47334,mrc-2-000004,"{'answer_start': [142], 'text': ['3점의 그릇']}"


In [23]:
len(gpt_train_df)

3547

In [24]:
# gpt 증강 데이터 csv 파일로 저장
gpt_train_df.to_csv("your_gpt_save_path.csv")